In [1]:
import pandas as pd
import etltools.etltools as etl

In [2]:
# modulo propio para esta ETL
# utilizar para cargar datos en BBDD
etl.etl()

        date state  positive  deathConfirmed
0 2021-03-07    AK   56886.0             NaN


In [40]:
# creamos el df desde la tabla de BBDD
engine = etl.connect_db()
conn = engine.connect()
df = pd.read_sql_table('clean_historic_values_all_states', conn, schema='public')
engine.dispose()
df

,index,date,state,positive,deathConfirmed
0,26,2021-03-07,MO,480643.0,NaN
1,29,2021-03-07,MT,100914.0,NaN
2,30,2021-03-07,NC,872176.0,10169.0
3,32,2021-03-07,NE,203026.0,NaN
4,33,2021-03-07,NH,76861.0,NaN
...,...,...,...,...,...
107269,20761,2020-01-26,MA,NaN,NaN
107270,20763,2020-01-25,MA,NaN,NaN
107271,20765,2020-01-24,MA,NaN,NaN
107272,20767,2020-01-23,MA,NaN,NaN


,index,date,state,positive,deathConfirmed
0,26,2021-03-07,MO,480643.0,NaN
1,29,2021-03-07,MT,100914.0,NaN
2,30,2021-03-07,NC,872176.0,10169.0
3,32,2021-03-07,NE,203026.0,NaN
4,33,2021-03-07,NH,76861.0,NaN
...,...,...,...,...,...
107269,20761,2020-01-26,MA,NaN,NaN
107270,20763,2020-01-25,MA,NaN,NaN
107271,20765,2020-01-24,MA,NaN,NaN
107272,20767,2020-01-23,MA,NaN,NaN


In [33]:
#!pip install pandas-profiling
#!pip install ydata-profiling
!pip install master.zip

Processing ./master.zip
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement scipy<1.10,>=1.4.1 (from ydata-profiling) (from versions: none)
ERROR: No matching distribution found for scipy<1.10,>=1.4.1


In [39]:
!conda install plotly

Retrieving notices: ...working... ERROR conda.notices.fetch:get_channel_notice_response(63): Request error <HTTPSConnectionPool(host='conda.anaconda.org', port=443): Max retries exceeded with url: /conda-forge/notices.json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fe34ef24040>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))> for channel: conda-forge url: https://conda.anaconda.org/conda-forge/notices.json
done

CondaHTTPError: HTTP 000 CONNECTION FAILED for url <https://conda.anaconda.org/conda-forge/linux-64/current_repodata.json>
Elapsed: -

An HTTP error occurred when trying to retrieve this URL.
HTTP errors are often intermittent, and a simple retry will get you on your way.
'https//conda.anaconda.org/conda-forge/linux-64'




In [36]:
import plotly.express as px

SyntaxError: invalid syntax (2265326421.py, line 1)

In [32]:
!wget https://github.com/ydataai/ydata-profiling/archive/refs/heads/master.zip

--2023-02-19 02:34:00--  https://github.com/ydataai/ydata-profiling/archive/refs/heads/master.zip
Resolving github.com (github.com)... failed: Temporary failure in name resolution.
wget: unable to resolve host address ‘github.com’


In [41]:
!ping 8.8.8.8

76795.57s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


/bin/bash: line 1: ping: command not found


In [ ]:


    original_df = extract(DATASET_URL)
    original_data_table_name = "original_historic_values"

    clean_df = transform(original_df)
    clean_table_name = "clean_historic_values_all_states"
    print(clean_df.head(1).to_string())

    load_data_to_db(original_df, original_data_table_name, db_engine)
    load_data_to_db(clean_df, clean_table_name, db_engine)



In [ ]:
# un truco para optimizar el flujo

def read_sql_tmpfile(query, db_engine):
    with tempfile.TemporaryFile() as tmpfile:
        copy_sql = "COPY ({query}) TO STDOUT WITH CSV {head}".format(
           query=query, head="HEADER"
        )
        conn = db_engine.raw_connection()
        cur = conn.cursor()
        cur.copy_expert(copy_sql, tmpfile)
        tmpfile.seek(0)
        df = pandas.read_csv(tmpfile)
        return df